In [1]:
%load_ext autoreload
%autoreload 2
import sys
import os
root_path = '../../'
sys.path.append(os.path.join(root_path, 'code', 'core'))


from ranking_utils import RankingUtils, Ranking
from mallows import *
from ws_ranking import WeakSupRanking
import numpy as np

from synth_exp_utils import generate_true_rankings, sample_mallows_LFs, estimate_theta, get_pair_wise_dists
import pickle

## Generate samples with Mallows model (below is about Mallow's model described by ChatGPT)

Mallows' model is a probabilistic model used for ranking and permutation problems. It is named after Colin L. Mallows, who introduced it in 1957. The model is designed to generate permutations of a set of items with a preference for permutations that are closer to a central ranking.

### Key Concepts of Mallows' Model

1. **Central Ranking ($\pi_0$)**: The model assumes a central or reference ranking $\pi_0$, which represents the most likely order of the items.

2. **Dispersion Parameter ($\theta$)**: A parameter $\theta$ controls the concentration of the distribution around the central ranking. When $\theta = 0$, all permutations are equally likely. As $\theta$ increases, the distribution becomes more concentrated around the central ranking.

3. **Kendall's Tau Distance**: The model often uses Kendall's tau distance to measure the dissimilarity between a given permutation $\pi$ and the central ranking $\pi_0$. Kendall's tau distance is the number of pairwise disagreements between two rankings.

### Probability Distribution

The probability of a permutation $\pi$ given the central ranking $\pi_0$ and the dispersion parameter $\theta$ is defined as:

$$ P(\pi | \pi_0, \theta) = \frac{1}{Z(\theta)} \exp(-\theta d(\pi, \pi_0)) $$

where:
- $d(\pi, \pi_0)$ is the Kendall's tau distance between $\pi$ and $\pi_0$.
- $Z(\theta)$ is a normalization constant (partition function) ensuring that the probabilities sum to one. It is given by:
$$ Z(\theta) = \sum_{\pi} \exp(-\theta d(\pi, \pi_0)) $$


### Parameter setup for LFs with Mallows' Model
- Each LF can be represented a Mallows' model with a parameter **Dispersion Parameter ($\theta$)**. Here we assume each LF is centered, i.e. knows the true center $\pi_0$.
- To see the effectiveness of our label model, we vary noiseness with dispersion parameters

In [2]:
num_lfs = 6

# Sample true dispersion parameters
theta_star = np.zeros(num_lfs)
half = int((num_lfs)/2)
theta_star[:half] = np.random.uniform(0.1,0.2,half) # Weak dispersion --> less noisy
theta_star[half:] = np.random.uniform(2,5,half) # Strong dispersion --> more noisy
np.sort(theta_star)
print(theta_star)
np.random.shuffle(theta_star)

[0.15488135 0.17151894 0.16027634 3.63464955 3.2709644  3.93768234]


### Generate synthetic data from Mallows' model

In [3]:

d = 3 # the number of items in each example
n = 1000 # the number of samples

r_utils = RankingUtils(d) # Utility object
Y = generate_true_rankings(d,n) # True rankings
print(Y[:3])

# Note that we use Ranking class defined in
# https://github.com/SprocketLab/universalizing-weak-supervision/blob/master/code/core/ranking_utils.py



[[0, 2, 1], [2, 1, 0], [0, 1, 2]]


In [4]:
# Instead of generating samples randomly, suppose we have ranking data.
# Here we just extract data from Y to simulate this
Y_rankings = [y.permutation for y in Y]
print('Naive type ranking list', Y_rankings[:3], type(Y_rankings[0]))

# Then, it is required to convert them to Ranking Objects to use UWS pipeline
# Convert back by wrapping rank by Ranking object
Y = [Ranking(rank) for rank in Y_rankings]
print('Ranking object list', Y[:3], type(Y[0]))

Naive type ranking list [[0, 2, 1], [2, 1, 0], [0, 1, 2]] <class 'list'>
Ranking object list [[0, 2, 1], [2, 1, 0], [0, 1, 2]] <class 'ranking_utils.Ranking'>


### Sample weak labels from LFs (defined by Mallows' model)

In [5]:
L = sample_mallows_LFs(Y, num_lfs, theta_star) # list[list[list[Ranking]]]
print(L[0]) # 3 items, 6 LFs

[[2, 0, 1], [1, 2, 0], [0, 2, 1], [0, 2, 1], [2, 0, 1], [0, 2, 1]]


In [6]:
# Instead of generating samples randomly, suppose we have weak ranking labels.
# Here we just extract data from L to simulate this.
L_rankings = []
for lf_rankings in L:
    naive_lf_rankings = []
    for one_lf_ranking in lf_rankings:
        naive_lf_rankings.append(one_lf_ranking.permutation)
    L_rankings.append(naive_lf_rankings)
print('Naive type LF rankings', L_rankings[0], type(L_rankings[0][0]))

# Then, it is required to convert them to Ranking Objects to use UWS pipeline
# Convert back by wrapping rank by Ranking object
L = []
for lf_rankings in L_rankings:
    Ranking_lf_rankings = []
    for one_lf_ranking in lf_rankings:
        Ranking_lf_rankings.append(Ranking(one_lf_ranking))
    L.append(Ranking_lf_rankings)
print('Ranking object list LF rankings', L[0], type(L[0][0]))

Naive type LF rankings [[2, 0, 1], [1, 2, 0], [0, 2, 1], [0, 2, 1], [2, 0, 1], [0, 2, 1]] <class 'list'>
Ranking object list LF rankings [[2, 0, 1], [1, 2, 0], [0, 2, 1], [0, 2, 1], [2, 0, 1], [0, 2, 1]] <class 'ranking_utils.Ranking'>


### Learn label model

In [8]:
label_model = WeakSupRanking(r_utils)
conf = {"train_method":"median_triplet_opt"}
label_model.train(conf, L, num_lfs) # This process estimates theta
label_model.thetas

array([0.99635107, 0.92735845, 3.50610817, 3.75047884, 0.90003872,
       3.44150772])

### Generate aggregated pseudolabels

In [9]:
mv_conf = {"train_method":"median_triplet_opt", "inference_rule": "kemeny"}
Y_mv = label_model.infer_ranking(mv_conf, L)

uws_conf = {"train_method":"median_triplet_opt", "inference_rule": "weighted_kemeny"}
Y_uws = label_model.infer_ranking(uws_conf, L)

In [10]:
mv_kt_dist = r_utils.mean_kt_distance(Y_mv, Y)
uws_kt_dist = r_utils.mean_kt_distance(Y_uws, Y)
print('mv_kt_dist:', mv_kt_dist, 'uws_kt_dist:', uws_kt_dist)

mv_kt_dist: 0.07966666666666666 uws_kt_dist: 0.001
